In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from lore_sa.dataset import TabularDataset

In [3]:
dataset = TabularDataset.from_csv('test/resources/adult.csv')
dataset.df.dropna(inplace = True)
dataset.df

2023-07-28 11:21:35,733 root         INFO     test/resources/adult.csv file imported


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [4]:
dataset.descriptor

{'numeric': {'age': {'index': 0,
   'min': 17,
   'max': 90,
   'mean': 38.58164675532078,
   'std': 13.640432553581341,
   'median': 37.0,
   'q1': 28.0,
   'q3': 48.0},
  'fnlwgt': {'index': 2,
   'min': 12285,
   'max': 1484705,
   'mean': 189778.36651208502,
   'std': 105549.97769702224,
   'median': 178356.0,
   'q1': 117827.0,
   'q3': 237051.0},
  'education-num': {'index': 4,
   'min': 1,
   'max': 16,
   'mean': 10.0806793403151,
   'std': 2.5727203320673877,
   'median': 10.0,
   'q1': 9.0,
   'q3': 12.0},
  'capital-gain': {'index': 10,
   'min': 0,
   'max': 99999,
   'mean': 1077.6488437087312,
   'std': 7385.292084840338,
   'median': 0.0,
   'q1': 0.0,
   'q3': 0.0},
  'capital-loss': {'index': 11,
   'min': 0,
   'max': 4356,
   'mean': 87.303829734959,
   'std': 402.9602186489998,
   'median': 0.0,
   'q1': 0.0,
   'q3': 0.0},
  'hours-per-week': {'index': 12,
   'min': 1,
   'max': 99,
   'mean': 40.437455852092995,
   'std': 12.347428681731843,
   'median': 40.0,
   

In [5]:
dataset.df.drop(['fnlwgt', 'education-num'], inplace=True, axis=1)
dataset.update_descriptor()
dataset.descriptor

{'numeric': {'age': {'index': 0,
   'min': 17,
   'max': 90,
   'mean': 38.437901995888865,
   'std': 13.134664776855985,
   'median': 37.0,
   'q1': 28.0,
   'q3': 47.0},
  'capital-gain': {'index': 8,
   'min': 0,
   'max': 99999,
   'mean': 1092.0078575691268,
   'std': 7406.346496683503,
   'median': 0.0,
   'q1': 0.0,
   'q3': 0.0},
  'capital-loss': {'index': 9,
   'min': 0,
   'max': 4356,
   'mean': 88.37248856176646,
   'std': 404.29837048637575,
   'median': 0.0,
   'q1': 0.0,
   'q3': 0.0},
  'hours-per-week': {'index': 10,
   'min': 1,
   'max': 99,
   'mean': 40.93123798156621,
   'std': 11.979984229274882,
   'median': 40.0,
   'q1': 40.0,
   'q3': 45.0}},
 'categoric': {'workclass': {'index': 1,
   'distinct_values': ['State-gov',
    'Self-emp-not-inc',
    'Private',
    'Federal-gov',
    'Local-gov',
    'Self-emp-inc',
    'Without-pay'],
   'count': {'State-gov': 1279,
    'Self-emp-not-inc': 2499,
    'Private': 22286,
    'Federal-gov': 943,
    'Local-gov': 2067

In [6]:
from lore_sa.encoder_decoder import OneHotEnc
encoder = OneHotEnc(dataset.descriptor)

encoded = []
for x in dataset.df.iloc:
    
    
    encoded.append(encoder.encode(x.values))
    
    


C:\Users\spine\Kode\Princess\LORE_sa\lore_sa\encoder_decoder\one_hot_enc.py:80: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if type is "categoric":


In [7]:
encoder.get_encoded_features()

{0: 'age',
 1: 'workclass=State-gov',
 2: 'workclass=Self-emp-not-inc',
 3: 'workclass=Private',
 4: 'workclass=Federal-gov',
 5: 'workclass=Local-gov',
 6: 'workclass=Self-emp-inc',
 7: 'workclass=Without-pay',
 8: 'education=Bachelors',
 9: 'education=HS-grad',
 10: 'education=11th',
 11: 'education=Masters',
 12: 'education=9th',
 13: 'education=Some-college',
 14: 'education=Assoc-acdm',
 15: 'education=7th-8th',
 16: 'education=Doctorate',
 17: 'education=Assoc-voc',
 18: 'education=Prof-school',
 19: 'education=5th-6th',
 20: 'education=10th',
 21: 'education=Preschool',
 22: 'education=12th',
 23: 'education=1st-4th',
 24: 'marital-status=Never-married',
 25: 'marital-status=Married-civ-spouse',
 26: 'marital-status=Divorced',
 27: 'marital-status=Married-spouse-absent',
 28: 'marital-status=Separated',
 29: 'marital-status=Married-AF-spouse',
 30: 'marital-status=Widowed',
 31: 'occupation=Adm-clerical',
 32: 'occupation=Exec-managerial',
 33: 'occupation=Handlers-cleaners',
 3

In [104]:
encoded_df =  pd.DataFrame(encoded, columns = [encoder.encoded_features[i] for i in range(len(encoded[0]))])
encoded_df

KeyError: 57

In [ ]:
# random generation

x = dataset.df.iloc[0].to_dict()

from lore_sa.neighgen.random import RandomGenerator

gen = RandomGenerator()

gen.generate(x,5, features_domain = {'workclass':list(dataset.df['workclass'].unique())})

In [ ]:
dataset_encoded.size


In [ ]:
dataset.df.size

In [ ]:
encoder = MyTargetEnc()

In [ ]:
#dataset encoded only for the user-specified features
dataset.class_name = "capital-gain"
df_enc = encoder.encode(dataset,['workclass','education','occupation','race','sex'], target = "capital-gain")
df_enc

In [ ]:
print (encoder)

#dataset back in his original format
encoder.decode(dataset)

In [5]:
from lore_sa.rule import Expression
import operator

exp = Expression("test", operator.gt,10)

print(exp)

test > 10


In [ ]:
import operator

operator.ge